In [ ]:
!pip install torchinfo

In [ ]:
from typing import Dict, Tuple
import json
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.optim as optim
import torch.utils.data
import numpy as np
import torch.optim as optim
import torch.utils.data
import numpy as np

from torch.optim.lr_scheduler import ReduceLROnPlateau


from torchinfo import summary

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# VAE Baseline - Direct Prediction from input string

## Data Preprocessing

In [ ]:
def save_dict_json(word2idx: Dict[str, int], filename: str) -> None:
    """
    Save the word2idx dictionary to a JSON file.

    Parameters:
    - word2idx (Dict[str, int]): The word-to-index mapping dictionary.
    - filename (str): The name of the file where the dictionary will be saved.
    """

    with open(filename, "w") as f:
        json.dump(word2idx, f)


def load_dict_json(filename: str) -> Dict[str, int]:
    """
    Load the word2idx dictionary from a JSON file.

    Parameters:
    - filename (str): The name of the file from which the dictionary will be loaded.

    Returns:
    - Dict[str, int]: The loaded word-to-index mapping dictionary.
    """

    with open(filename, "r") as f:
        word2idx = json.load(f)
    return word2idx

In [ ]:
def transform_dict_values(input_dict: dict) -> dict:
    """
    Transforms the values of a dictionary by applying the formula 2 * i + 3,
    where i is the original value.

    Parameters:
    input_dict (dict): A dictionary with integer values.

    Returns:
    dict: A new dictionary with transformed values.
    """
    transformed_dict = {}
    for key, value in input_dict.items():
        # Assuming the values can be converted to integers
        transformed_value = 1 * int(value)
        # + 3
        transformed_dict[key] = transformed_value

    return transformed_dict

In [ ]:
ct2idx = load_dict_json(r"/content/drive/MyDrive/Project_MingyuZhong_Ver.11.10/Project/data/cell_type_word2idx.json")
pert2idx = load_dict_json(r"/content/drive/MyDrive/Project_MingyuZhong_Ver.11.10/Project/data/sm_name_word2idx.json")
print((ct2idx))
print((pert2idx))

ct2idx = transform_dict_values(ct2idx)
pert2idx = transform_dict_values(pert2idx)
print((ct2idx))
print((pert2idx))

{'B cells': 0, 'Myeloid cells': 1, 'NK cells': 2, 'T cells CD4+': 3, 'T cells CD8+': 4, 'T regulatory cells': 5}
{'5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6-yl)pyrimidin-2-amine': 0, 'ABT-199 (GDC-0199)': 1, 'ABT737': 2, 'AMD-070 (hydrochloride)': 3, 'AT 7867': 4, 'AT13387': 5, 'AVL-292': 6, 'AZ628': 7, 'AZD-8330': 8, 'AZD3514': 9, 'AZD4547': 10, 'Alogliptin': 11, 'Alvocidib': 12, 'Amiodarone': 13, 'Atorvastatin': 14, 'Azacitidine': 15, 'BAY 61-3606': 16, 'BAY 87-2243': 17, 'BI-D1870': 18, 'BMS-265246': 19, 'BMS-387032': 20, 'BMS-536924': 21, 'BMS-777607': 22, 'BX 912': 23, 'Belinostat': 24, 'Bosutinib': 25, 'Buspirone': 26, 'CC-401': 27, 'CEP-18770 (Delanzomib)': 28, 'CEP-37440': 29, 'CGM-097': 30, 'CGP 60474': 31, 'CHIR-99021': 32, 'Cabozantinib': 33, 'Canertinib': 34, 'Ceritinib': 35, 'Chlorpheniramine': 36, 'Clemastine': 37, 'Clomipramine': 38, 'Clotrimazole': 39, 'Colchicine': 40, 'Colforsin': 41, 'Crizotinib': 42, 'Dabrafenib': 43, 'Dactolisib': 44, 'Dasatinib': 45, 'Decita

## Dataset Construct

In [ ]:
def Obtain_VAE_data(parquet_file_path: str) -> np.ndarray:
    # Load the Parquet file into a DataFrame
    df = pd.read_parquet(parquet_file_path)

    # The name of the column to transform
    column_to_transform = "cell_type"

    # Replace the values in the column based on the dictionary
    df[column_to_transform] = df[column_to_transform].map(ct2idx)

    # The name of the column to transform
    column_to_transform = "sm_name"

    # Replace the values in the column based on the dictionary
    df[column_to_transform] = df[column_to_transform].map(pert2idx)

    df.drop(columns=["sm_lincs_id", "SMILES"], inplace=True)
    # ,'control'

    return df.values

In [ ]:
# Replace with your actual file path
training_path = r"/content/drive/MyDrive/Project_MingyuZhong_Ver.11.10/Project/data/data_train.parquet"
training_data = Obtain_VAE_data(training_path)
print(training_data)

[[5 103 False ... 1.1474819425496732 -0.0013708856043735814
  -0.5387391267629659]
 [4 44 False ... 0.14008628240333457 -0.27597584820022547
  0.30352137303390303]
 [4 5 False ... 0.3415228027084855 -0.48192001204442475
  0.007458666648073425]
 ...
 [4 82 False ... -0.060238740601505275 0.8770697568907733
  1.0929688729439277]
 [1 12 False ... -0.8517764052953564 0.04442567270763101
  -0.497355486141355]
 [2 92 False ... -0.1867229415185727 0.443236562810196 0.5938353783439132]]


In [ ]:
class ParquetDataset(Dataset):
    def __init__(self, filename):
        self.data = Obtain_VAE_data(filename)
        # print(self.data.head(2))
        # print(self.data.shape)
        self.data = torch.tensor(self.data, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data_row = self.data[idx]
        tuple_pair = data_row[:2]  # First two columns
        Gene_DEVs = data_row[2:]  # Remaining columns
        return Gene_DEVs, tuple_pair

## Metric Definition

In [ ]:
def mean_rowwise_rmse(output, target):
    """
    Compute Mean Rowwise Root Mean Squared Error.

    Args:
        output (torch.Tensor): The model's output.
        target (torch.Tensor): The target values.

    Returns:
        torch.Tensor: The mean rowwise RMSE.
    """
    mse_per_row = torch.mean((output - target) ** 2, dim=1)
    rmse_per_row = torch.sqrt(mse_per_row)
    return torch.mean(rmse_per_row)

## Model Definitions

In [ ]:
class VAE(nn.Module):
    """
    Variational Autoencoder (VAE) for predicting 18,000 Gene DEV based on (cell type, Perturbagen) tuple pair.
    """

    class Encoder(nn.Module):
        """
        Encoder network for VAE.
        """

        def __init__(self, num_features: int, dropout_rate: float):
            super().__init__()
            # Adjusting layer dimensions as per the specified architecture
            self.fc1 = nn.Linear(num_features, 1024)
            self.bn1 = nn.BatchNorm1d(1024)
            self.attention1 = nn.MultiheadAttention(1024, num_heads=4, batch_first=True)

            self.fc2 = nn.Linear(1024, 128)
            self.bn2 = nn.BatchNorm1d(128)
            self.attention2 = nn.MultiheadAttention(128, num_heads=4, batch_first=True)

            self.fc3 = nn.Linear(128, 16)
            self.bn3 = nn.BatchNorm1d(16)
            self.attention3 = nn.MultiheadAttention(16, num_heads=4, batch_first=True)

            # Separate outputs for mean and standard deviation
            self.fc_mu = nn.Linear(16, 2)
            self.fc_log_var = nn.Linear(16, 2)
            self.dropout = nn.Dropout(dropout_rate)
            self.act_fn = nn.LeakyReLU()

        def forward(self, x: torch.Tensor) -> (torch.Tensor, torch.Tensor):
            x = self.act_fn(self.bn1(self.fc1(x)))
            x = self.dropout(x)

            # resid = x
            # x, _ = self.attention1(x, x, x)
            # x = x + resid
            # x = self.dropout(x)

            x = self.act_fn(self.bn2(self.fc2(x)))
            x = self.dropout(x)

            # resid = x
            # x, _ = self.attention2(x, x, x)
            # x = x + resid
            # x = self.dropout(x)

            x = self.act_fn(self.bn3(self.fc3(x)))
            x = self.dropout(x)

            # resid = x
            # x, _ = self.attention3(x, x, x)
            # x = x + resid
            # x = self.dropout(x)

            mu = self.fc_mu(x)
            log_var = self.fc_log_var(x)
            return mu, log_var

    class Decoder(nn.Module):
        """
        Decoder network for VAE.
        """

        def __init__(self, num_features: int, dropout_rate: float):
            super().__init__()
            # Reversing the architecture of the encoder
            self.fc1 = nn.Linear(2, 16)
            self.bn1 = nn.BatchNorm1d(16)
            self.fc2 = nn.Linear(16, 128)
            self.bn2 = nn.BatchNorm1d(128)
            self.fc3 = nn.Linear(128, 1024)
            self.bn3 = nn.BatchNorm1d(1024)
            self.fc4 = nn.Linear(
                1024, num_features
            )  # Output layer matching Gene DEV dimension
            self.dropout = nn.Dropout(dropout_rate)
            self.act_fn = nn.LeakyReLU()
            self.attention1 = nn.MultiheadAttention(16, num_heads=4, batch_first=True)
            self.attention2 = nn.MultiheadAttention(128, num_heads=4, batch_first=True)
            self.attention3 = nn.MultiheadAttention(1024, num_heads=4, batch_first=True)

        def forward(self, z: torch.Tensor) -> torch.Tensor:
            z = self.act_fn(self.bn1(self.fc1(z)))
            z = self.dropout(z)

            # resid = z
            # z, _ = self.attention1(z, z, z)
            # z = z + resid
            # z = self.dropout(z)

            z = self.act_fn(self.bn2(self.fc2(z)))
            z = self.dropout(z)

            # resid = z
            # z, _ = self.attention2(z, z, z)
            # z = z + resid
            # z = self.dropout(z)

            z = self.act_fn(self.bn3(self.fc3(z)))
            z = self.dropout(z)

            # resid = z
            # z, _ = self.attention3(z, z, z)
            # z = z + resid
            # z = self.dropout(z)

            reconstruction = self.fc4(z)
            return reconstruction

    def __init__(self, num_features: int, dropout_rate: float):
        super(VAE, self).__init__()
        self.encoder = self.Encoder(num_features, dropout_rate)
        self.decoder = self.Decoder(num_features, dropout_rate)

    def reparameterize(self, mu: torch.Tensor, log_var: torch.Tensor) -> torch.Tensor:
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def encode(self, x: torch.Tensor) -> torch.Tensor:
        mu, log_var = self.encoder(x)
        z = self.reparameterize(mu, log_var)
        return z

    def decode(self, z: torch.Tensor) -> torch.Tensor:
        x_reconstructed = self.decoder(z)
        return x_reconstructed

    def forward(self, x: torch.Tensor) -> (torch.Tensor, torch.Tensor, torch.Tensor):
        mu, log_var = self.encoder(x)
        z = self.reparameterize(mu, log_var)
        x_reconstructed = self.decoder(z)
        return x_reconstructed, mu, log_var

In [ ]:
def vae_loss(
    x_decoded_mean: torch.Tensor,
    x: torch.Tensor,
    z_mean: torch.Tensor,
    z_logvar: torch.Tensor,
    mu_t: torch.Tensor,
    std_t: torch.Tensor,
    device: torch.device,
) -> torch.Tensor:
    """
    Calculate the loss for the Variational Autoencoder.

    Parameters:
        x_decoded_mean (torch.Tensor): The decoded output of the VAE.
        x (torch.Tensor): The original input data.
        z_mean (torch.Tensor): Mean of the latent space.
        z_logvar (torch.Tensor): Log variance of the latent space.
        mu_t (torch.Tensor): Prior mean of the latent space distribution.
        std_t (torch.Tensor): Prior standard deviation of the latent space distribution.
        device (torch.device): The device (CPU or GPU) where the tensors are located.

    Returns:
        torch.Tensor: The calculated VAE loss.
    """
    # Reconstruction Loss
    recon_loss = F.mse_loss(x_decoded_mean, x, reduction="mean")

    # KL Divergence Loss
    covar_q = torch.exp(z_logvar).to(device)  # Variance of the learned distribution
    covar_p = torch.ones_like(covar_q).to(device) * std_t.pow(
        2
    )  # Variance of the prior distribution

    mu_adjusted = z_mean - mu_t.to(device).unsqueeze(1)  # Adjusting the mean

    # KL Divergence calculation
    matrix_sum = (
        torch.log(covar_p) - z_logvar - 1 + (mu_adjusted.pow(2) + covar_q) / covar_p
    )
    kl_div = 0.5 * torch.sum(matrix_sum, dim=1)
    kl_loss = torch.mean(kl_div)

    return recon_loss + kl_loss

In [ ]:
import torch
import torch.nn.functional as F


def train_np(
    model,
    train_loader,
    val_loader,
    n_epochs=100,
    lr=0.01,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
):
    # # Data loaders
    # train_loader = torch.utils.data.DataLoader(
    #     dataset=train_data, batch_size=batch_size, shuffle=True
    # )
    # val_loader = torch.utils.data.DataLoader(
    #     dataset=val_data, batch_size=batch_size, shuffle=False
    # )
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = ReduceLROnPlateau(
        optimizer, mode="min", factor=0.1, patience=128, verbose=True
    )

    best_val_loss = float("inf")
    # Training loop
    for epoch in range(1, n_epochs + 1):
        # Training mode
        model.train()

        train_loss, train_rmse = 0.0, 0.0

        for batch_idx, batch_x in enumerate(train_loader):
            data, mu_t = batch_x
            # batch_x[:, :-1], batch_x[:, -1]
            mu_t = torch.unsqueeze(mu_t, 1)
            mu_std = torch.tensor([1], dtype=torch.float32)
            data, mu_t, mu_std = data.to(device), mu_t.to(device), mu_std.to(device)

            # Forward pass
            output, mean, logvar = model(data)
            loss = vae_loss(output, data, mean, logvar, mu_t, mu_std, device)

            # Update
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_rmse += mean_rowwise_rmse(output, data).item()

        # Calculate average training loss for the epoch
        train_loss /= len(train_loader)
        train_rmse /= len(train_loader)

        # Validation mode
        model.eval()
        val_loss, val_rmse = 0.0, 0.0
        with torch.no_grad():
            for batch_x in val_loader:
                data, mu_t = batch_x
                # [:, :-1], batch_x[:, -1]
                mu_t = torch.unsqueeze(mu_t, 1)
                data, mu_t = data.to(device), mu_t.to(device)

                # Forward pass
                output, mean, logvar = model(data)
                loss = vae_loss(output, data, mean, logvar, mu_t, mu_std, device)

                val_loss += loss.item()
                val_rmse += mean_rowwise_rmse(output, data).item()

        # Calculate average validation loss for the epoch
        val_loss /= len(val_loader)
        val_rmse /= len(val_loader)
        print(
            f"Epoch {epoch} - Train Loss: {train_loss:.4f}, Train MRRMSE: {train_rmse:.4f}, Validation Loss: {val_loss:.4f}, Validation MRRMSE: {val_rmse:.4f}"
        )

        scheduler.step(val_loss)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
    return model

In [ ]:
Gene_DEVs, tuple_pair = next(train_data_iterator)

num_features = Gene_DEVs.shape[1]

NameError: ignored

## Model & Dataset Initialization

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

train_path = r"/content/drive/MyDrive/Project_MingyuZhong_Ver.11.10/Project/data/data_train.parquet"
valid_path = r"/content/drive/MyDrive/Project_MingyuZhong_Ver.11.10/Project/data/data_val.parquet"

train_dataset = ParquetDataset(train_path)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
valid_dataset = ParquetDataset(valid_path)
valid_dataloader = DataLoader(valid_dataset, batch_size=256, shuffle=False)

# Create an iterator from the dataloader
train_data_iterator = iter(train_dataloader)

# Fetch the next batch
Gene_DEVs, tuple_pair = next(train_data_iterator)

num_features = Gene_DEVs.shape[1]
dropout_rate = 0.19

# Instantiate the VAE model
vae_model = VAE(num_features=num_features, dropout_rate=dropout_rate)

# Generate the model summary
model_summary = summary(vae_model, input_size=(1, num_features))

print(model_summary)

cuda


TypeError: ignored

## Training

In [ ]:
vae = train_np(
    vae_model, train_loader=train_dataloader, val_loader=valid_dataloader, n_epochs=1000
)

Epoch 1 - Train Loss: 1730.7991, Train MRRMSE: 2.0135, Validation Loss: 2057.8450, Validation MRRMSE: 1.2323
Epoch 2 - Train Loss: 1698.1965, Train MRRMSE: 1.8813, Validation Loss: 1986.6821, Validation MRRMSE: 1.1171
Epoch 3 - Train Loss: 1719.8886, Train MRRMSE: 1.7535, Validation Loss: 1973.4066, Validation MRRMSE: 1.2047
Epoch 4 - Train Loss: 1684.1452, Train MRRMSE: 1.4933, Validation Loss: 1958.7358, Validation MRRMSE: 1.1042
Epoch 5 - Train Loss: 1664.0436, Train MRRMSE: 1.4092, Validation Loss: 1944.3069, Validation MRRMSE: 1.2056
Epoch 6 - Train Loss: 1637.8923, Train MRRMSE: 1.3229, Validation Loss: 1928.3425, Validation MRRMSE: 1.0562
Epoch 7 - Train Loss: 1642.6296, Train MRRMSE: 1.3229, Validation Loss: 1921.2729, Validation MRRMSE: 1.7954
Epoch 8 - Train Loss: 1590.9657, Train MRRMSE: 1.2047, Validation Loss: 1891.0306, Validation MRRMSE: 1.0933
Epoch 9 - Train Loss: 1613.3488, Train MRRMSE: 1.2868, Validation Loss: 1867.2418, Validation MRRMSE: 1.0014
Epoch 10 - Train Lo

## Test Evaluation

In [ ]:
from sympy import true


test_path = r"..\data\data_test.parquet"  # Path to the test dataset
test_dataset = ParquetDataset(test_path)
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=False)


def predict_with_vae(model, dataloader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch_x in dataloader:
            _, data = batch_x
            data = data.to(device)
            reconstructed_data = model.decode(data)

            predictions.append(reconstructed_data.cpu().numpy())

    return np.concatenate(predictions, axis=0)


# Load the true values for the test dataset
true_values = [data.numpy() for data, _ in test_dataloader]
true_values = np.array(true_values).squeeze(0)
print(true_values.shape)

# Make predictions
predictions = predict_with_vae(vae_model, test_dataloader, device)
print(predictions.shape)
# Calculate metrics
metric_result = mean_rowwise_rmse(
    torch.from_numpy(predictions), torch.from_numpy(true_values)
)
print({metric_result})

(71, 7818)
(71, 7818)
{tensor(4.9415)}
